In [ ]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Q4").getOrCreate()

In [3]:
from pyspark import SparkFiles
# Load in reviews.csv from S3 into a DataFrame
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.1/22-big-data/day_3/reviews.csv"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("reviews.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df.show(10)

+----------+--------------------+
|      date|         review_text|
+----------+--------------------+
|2010-07-05|- El artículo abo...|
|2010-07-05|El artículo prese...|
|2010-07-05|- El tema es muy ...|
|2010-07-05|"Se explica en fo...|
|2010-07-05|                null|
|2010-07-05|"Los autores desc...|
|2010-07-05|Este trabajo prop...|
|2010-07-05|"This paper aims ...|
|2010-07-05|The paper is well...|
|2010-07-05|Se realiza un tra...|
+----------+--------------------+
only showing top 10 rows



# What date had the highest number of reviews?

In [4]:
# Order review and date with higesest reviews
date_df = df.groupBy('date').agg({"date": "count"})
date_df.show()

+----------+-----------+
|      date|count(date)|
+----------+-----------+
|2010-07-05|        117|
|2013-07-05|         82|
|2014-07-05|        126|
|2015-07-05|         80|
+----------+-----------+



In [5]:
from pyspark.sql.functions import desc
date_df.orderBy(desc("count(date)")).show()

+----------+-----------+
|      date|count(date)|
+----------+-----------+
|2014-07-05|        126|
|2010-07-05|        117|
|2013-07-05|         82|
|2015-07-05|         80|
+----------+-----------+

